# Анализ твитов - обработка текста

Мы будем строить классификатор, который будет разделять текст на позитивные и негативные высказывания. Для этого мы воспользуемся уже размеченной базой. Загрузим данные для анализа

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

Populating the interactive namespace from numpy and matplotlib


In [ ]:
! pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 85.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=06b0bbe55763af621c161deb76bb66886a1eec8ec6428b7b5cb3ee97894b8ed2
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt



## Задание 1. Загрузка данных

Источник: https://drive.google.com/file/d/1wnj7wWvpCZiB8-kpFdV6WhAQSip-GSLV/view?usp=share_link

- Загрузите датафреймы positive.csv и negative.csv
- Объедините датафреймы с помощью pd.concat() в один датафрейм;
- Убедитесь, что в новом датафрейме индексация сквозная и без повторов;
- Переименуйте столбцы датафрейма;
```
df.rename(columns = {0 : 'id', 1 : 'date', 2: 'name', 3: 'text', 4: 'positive', 5: 'rep', 6: 'rtv', 7: 'fav',
    8: 'total_count', 9: 'fol', 10: 'friends', 11: 'list_count'}, inplace = True)
```
- Выведите информацию об общем количестве полученных твитов, сколько из них негативных, сколько позитивных.

In [ ]:
positive = pd.read_csv('/content/positive.csv', sep=';', header=None, encoding='latin-1', engine='python', on_bad_lines='skip')
positive.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee Ñ Ð¾ÑÑ Ñ Ð¸ ÑÐºÐ¾Ð»Ð¾ÑÐ°, Ð½...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"ÐÐ°, Ð²ÑÐµ-ÑÐ°ÐºÐ¸ Ð¾Ð½ Ð½ÐµÐ¼Ð½Ð¾Ð³Ð¾ Ð¿Ð¾...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: ÐÑ ÑÑ Ð¸Ð´Ð¸Ð¾ÑÐºÐ°) Ñ Ð¸...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""ÐÑÐ¾ ÑÐ¾ Ð² ÑÐ³Ð»Ñ ÑÐ¸Ð...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant ÐÐ¾Ñ ÑÑÐ¾ Ð·Ð½Ð°ÑÐ¸Ñ ÑÑ...,1,0,0,0,597,16,23,1


In [ ]:
negative = pd.read_csv('/content/negative.csv', sep=';', header=None, encoding='latin-1', engine='python', on_bad_lines='skip')
negative.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,Ð½Ð° ÑÐ°Ð±Ð¾ÑÐµ Ð±ÑÐ» Ð¿Ð¾Ð»Ð½ÑÐ¹ Ð¿Ð¸Ð´Ð´...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,ÐÐ¾Ð»Ð»ÐµÐ³Ð¸ ÑÐ¸Ð´ÑÑ ÑÑÐ±ÑÑÑÑ Ð² Ur...,-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post ÐºÐ°Ðº Ð³Ð¾Ð²Ð¾ÑÑÑ Ð¾Ð±ÐµÑÐ°Ð...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,ÐÐµÐ»Ð°Ñ Ñ Ð¾ÑÐ¾ÑÐµÐ³Ð¾ Ð¿Ð¾Ð»ÑÑÐ° Ð¸ Ñ...,-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,ÐÐ±Ð½Ð¾Ð²Ð¸Ð» Ð·Ð° ÐºÐ°ÐºÐ¸Ð¼-ÑÐ¾ Ð»ÐµÑÐ¸Ð¼...,-1,0,0,0,35,17,34,0


In [ ]:
df = pd.concat([positive, negative], ignore_index=True)
df.rename(columns = {0 : 'id', 1 : 'date', 2: 'name', 3: 'text', 4: 'positive', 5: 'rep', 6: 'rtv', 7: 'fav',
  8: 'total_count', 9: 'fol' , 10: 'friends', 11: 'list_count'}, inplace = True)
df

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee Ñ Ð¾ÑÑ Ñ Ð¸ ÑÐºÐ¾Ð»Ð¾ÑÐ°, Ð½...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"ÐÐ°, Ð²ÑÐµ-ÑÐ°ÐºÐ¸ Ð¾Ð½ Ð½ÐµÐ¼Ð½Ð¾Ð³Ð¾ Ð¿Ð¾...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: ÐÑ ÑÑ Ð¸Ð´Ð¸Ð¾ÑÐºÐ°) Ñ Ð¸...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""ÐÑÐ¾ ÑÐ¾ Ð² ÑÐ³Ð»Ñ ÑÐ¸Ð...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant ÐÐ¾Ñ ÑÑÐ¾ Ð·Ð½Ð°ÑÐ¸Ñ ÑÑ...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,ÐÐ¾ Ð½Ðµ ÐºÐ°Ð¶Ð´ÑÐ¹ Ñ Ð¾ÑÐµÑ ÑÑÐ¾ ÑÐ¾ ...,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,ÑÐºÑÑÐ°Ñ ÑÐ°Ðº :-( ÑÐ¾Ð»ÑÐºÐ¾ @taaannya...,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"ÐÐ¾Ñ Ð¸ Ð² ÑÐºÐ¾Ð»Ñ, Ð² Ð³Ð¾Ð²Ð½Ð¾ ÑÑÐ¾ ...",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Ð¢Ð°ÑÑÐ¸ÑÐ»Ñ, Ð½Ð...",-1,0,1,0,2516,187,265,0


In [ ]:
print(f"Общее количество твитов {df.shape[0]}")
print(f"Позитивные твиты {len(df[df["positive"]== 1])}")
print(f"Негативные твиты {len(df[df["positive"]== -1])}")

Общее количество твитов 226834
Позитивные твиты 114911
Негативные твиты 111923


## Задание 2. Очистка данных

Произведите очистку данных. По результатам очистки выведите на экран следующую информацию:
- Общее количество слов перед удалением слов, встречающихся 1 раз;
- Количество слов, встречающихся 1 раз;
- Итоговое количество слов;
- Количество пустых твитов;
- Из них позитивных твитов;
- Количество твитов после удаления пустых.


In [ ]:
# 1. Приведите весь текст к строчным буквам
df['text'] = df['text'].str.lower()
df['text']

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"да, все-таки он немного похож на него. но мой ..."
2,rt @katiacheh: ну ты идиотка) я испугалась за ...
3,"rt @digger2912: ""кто то в углу сидит и погибае..."
4,@irina_dyshkant вот что значит страшилка :d\nн...
...,...
226829,но не каждый хочет что то исправлять:( http://...
226830,скучаю так :-( только @taaannyaaa вправляет мо...
226831,"вот и в школу, в говно это идти уже надо("
226832,"rt @_them__: @lisaberoud тауриэль, не грусти :..."


In [ ]:
# 2. Оставьте в тексте только русские слова, удалив числа, знаки препинания, специальные символы и слова написанные латиницу

df["text"] = df["text"].str.replace(r"[^А-Яа-я]", " ", regex=True)
df['text']

,text
0,хоть я и школота но поверь у на...
1,да все таки он немного похож на него но мой ...
2,ну ты идиотка я испугалась за ...
3,кто то в углу сидит и погибае...
4,вот что значит страшилка но...
...,...
226829,но не каждый хочет что то исправлять ...
226830,скучаю так только вправляет мо...
226831,вот и в школу в говно это идти уже надо
226832,тауриэль не грусти ...


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# 3. Разбейте тексты на слова с помощью word_tokenize

from nltk.tokenize import word_tokenize
df["text"] = list(map(word_tokenize, df["text"]))
df["text"].head()

,text
0,"[хоть, я, и, школота, но, поверь, у, нас, то, ..."
1,"[да, все, таки, он, немного, похож, на, него, ..."
2,"[ну, ты, идиотка, я, испугалась, за, тебя]"
3,"[кто, то, в, углу, сидит, и, погибает, от, гол..."
4,"[вот, что, значит, страшилка, но, блин, посмот..."


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# 4. Удалите стоп-слова

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
len(russian_stopwords)

151

In [ ]:
# удаляем стоп-слова

def remove_stopwords(tokens):
    return [token for token in tokens if token not in russian_stopwords]

df['text'] = df['text'].apply(remove_stopwords)
df['text'].head()

,text
0,"[школота, поверь, самое, общество, профилирующ..."
1,"[таки, немного, похож, мальчик, равно]"
2,"[идиотка, испугалась]"
3,"[углу, сидит, погибает, голода, ещ, порции, вз..."
4,"[значит, страшилка, блин, посмотрев, части, со..."


In [ ]:
pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 62.9 MB/s eta 0:00:00


In [ ]:
# 5. Проведите лемматизацию
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")
def stemming(words):
    return [stemmer.stem(word) for word in words]
df["text"] = df["text"].apply(stemming)
df["text"].head()

,text
0,"[школот, повер, сам, обществ, профилир, предме..."
1,"[так, немн, похож, мальчик, равн]"
2,"[идиотк, испуга]"
3,"[угл, сид, погиба, голод, ещ, порц, взял, хот,..."
4,"[знач, страшилк, блин, посмотр, част, создаст,..."


In [ ]:
# 6. Удалите все слова, которые встречаются только 1 раз

from collections import Counter

all_words = [word for sublist in df['text'] for word in sublist]
word_counts = Counter(all_words)

df['text'] = df['text'].apply(
    lambda words: [word for word in words if word_counts[word] > 1]
)

df["text"].head()

,text
0,"[школот, повер, сам, обществ, предмет, тип]"
1,"[так, немн, похож, мальчик, равн]"
2,"[идиотк, испуга]"
3,"[угл, сид, погиба, голод, ещ, порц, взял, хот,..."
4,"[знач, страшилк, блин, посмотр, част, ощущен, ..."


In [ ]:
# 7. Удалите строчки с пустыми твитами

df = df[df['text'].str.len() > 0]
df

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"[школот, повер, сам, обществ, предмет, тип]",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"[так, немн, похож, мальчик, равн]",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,"[идиотк, испуга]",1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"[угл, сид, погиба, голод, ещ, порц, взял, хот,...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,"[знач, страшилк, блин, посмотр, част, ощущен, ...",1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,"[кажд, хочет, исправля]",-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,"[скуча, вправля, мозг, равн, скуча]",-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"[школ, говн, эт, идт]",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"[тауриэл, груст, обня]",-1,0,1,0,2516,187,265,0


In [ ]:
# 8. Какое количество твитов осталось. Сколько из них позитивных, сколько негативных?

total_tweets = len(df)
print(f"Общее количество твитов: {total_tweets}")

# Количество позитивных (1) и негативных (-1) твитов
positive_count = len(df[df['positive'] == 1])
negative_count = len(df[df['positive'] == -1])

print(f"Позитивных твитов: {positive_count}")
print(f"Негативных твитов: {negative_count}")

Общее количество твитов: 226010
Позитивных твитов: 114539
Негативных твитов: 111471


In [ ]:
df.to_csv('cleaned_tweets.csv', index=False, encoding='utf-8')

## Задание 3. Классика ML

1. Кодировать данные методом мешка слов.
2. Построить классификатор на основе логистической регрессии
3. Построить классификатор на основе случайного леса

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.positive, test_size=0.3, random_state=42)

len(y_train), y_train.sum(), len(y_test), y_test.sum()

NameError: name 'df' is not defined

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df.text, df.positive, test_size=0.3, random_state=42)

bow = CountVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    min_df=2
)

X_train_bow = bow.fit_transform(X_train)
X_test_bow  = bow.transform(X_test)

print(X_train_bow.shape)
print(X_test_bow.shape)

# глянем на фичи
feature_sample = bow.get_feature_names_out()[:25]
print("Примеры признаков:", feature_sample)


In [ ]:
# 2. Логистическая регрессия по мешку слов
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score

logreg = LogisticRegression(
    solver='liblinear',
    class_weight='balanced',
    random_state=42
)

logreg.fit(X_train_bow, y_train)
y_pred_lr = logreg.predict(X_test_bow)

acc_lr = accuracy_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr, pos_label=1)

print(f"[LogReg] Accuracy: {acc_lr:.3f} | F1(positive=1): {f1_lr:.3f}")
print(classification_report(y_test, y_pred_lr, digits=3))


In [ ]:
# 3. Случайный лес по тем же признакам
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced_subsample'
)

rf.fit(X_train_bow, y_train)
y_pred_rf = rf.predict(X_test_bow)

acc_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, pos_label=1)

print(f"[RandomForest] Accuracy: {acc_rf:.3f} | F1(positive=1): {f1_rf:.3f}")
print(classification_report(y_test, y_pred_rf, digits=3))
